# Projeto 2 - Módulo 1
Utilizar o banco da dados das flores de íris de Fischer para realizar as seguintes tarefas:
1. Utilizar a árvore de decisão de cada modelo para classificar novas flores.
2. Criar uma função de classificação para cada modelo, a partir de suas respectivas árvores de decisão. As árvores de decisão são especificadas na Seção Modelos.
3. Classificar todas as novas flores, por meio destas funções. Os dados das novas flores são especificados na seção Dados.
4. Definir o melhor modelo
5. Definir um modo de medir a performance de cada modelo, a partir das classificações feitas pelos modelos e da real classificação da espécie.
6. Qual tipo de flor possui mais erros em sua classificação? Considere cada um dos modelos.
7. Comparar a performance dos modelos.

## Importando os módulos que serão utilizados

In [37]:
from sklearn import datasets
import pandas as pd
import numpy as np
import plotly.express as px

## Importando do dataset das Íris

In [38]:
iris = datasets.load_iris()
dados = pd.DataFrame(data=np.c_[iris['data'], iris['target']],
                     columns=iris['feature_names']+['target'])


## Análise exploratória

In [39]:
dados.head()


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [40]:
dados.tail()


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
145,6.7,3.0,5.2,2.3,2.0
146,6.3,2.5,5.0,1.9,2.0
147,6.5,3.0,5.2,2.0,2.0
148,6.2,3.4,5.4,2.3,2.0
149,5.9,3.0,5.1,1.8,2.0


O dataset contém 150 oservações com quatro atributos cada, lagura e comprimento da sépala (*sepal width* e *sepal length*, respectivamente) e largura e comprimento da pétala (*petal width* e *petal length*, respectivamente) todos em centímetros (cm). O dataset vem também com o target, a classificaçao valores inteiros de 0, 1 e 2, sendo respectivamente 0 = setosa, 1 = versicolor, 2 = virginica.

In [41]:
dados.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sepal length (cm)  150 non-null    float64
 1   sepal width (cm)   150 non-null    float64
 2   petal length (cm)  150 non-null    float64
 3   petal width (cm)   150 non-null    float64
 4   target             150 non-null    float64
dtypes: float64(5)
memory usage: 6.0 KB


## Boxplot

In [42]:
dados.describe()


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333,1.000000
std,0.828066,0.435866,1.765298,0.762238,0.819232
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


In [43]:
fig_box = px.box(dados, y=["sepal length (cm)", "sepal width (cm)",
                 "petal length (cm)", "petal width (cm)"], points="all", color="target")
fig_box.show()


Declarando os nomes dos targets para utilizar mais tarde, pois o target do dataset são números inteiros de 0 a 2

In [44]:
target_names = ["setosa", "versicolor", "virginica"]


## Scatter Plot
Verificando o gráfico de dispersão de *sepal length* x *sepal width*, não é possível notar grande correlação entre os dois valores.

In [45]:
fig_scatter = px.scatter(dados, x="sepal length (cm)",
                         y="sepal width (cm)", color="target", title="Dispersão sepal length x sepal width")
fig_scatter.show()

Já no gáfico de dispersão *petal length* x *petal width*, podemos notar uma grande correlaçao entre os dois atributos.

In [46]:
fig_scatter = px.scatter(dados, x="petal length (cm)",
                         y="petal width (cm)", color="target", title="Dispersão petal length x petal width")
fig_scatter.show()


## Modelo de Árvore 1
A imagem da ávore pode ser encontrada no gitbook da proposta do projeto.

Declarando os nós em um python dictionary.

A árvore foi dividida em níveis, cima para baixo, esquerda para direita:
1. root
2. node_1
   * leaf
   * node
3. node_2
    * a
    * b
4. node_3
   * a
   * b
   * c
   * leaf
5. leafs

As *leafs* é onde os dados estão em sua classificação final, enquando nos nodes, os dados ainda passarão por algum processo de decisão e classificação.

In [47]:
nodes_tree_1 = {
    "root": {
        "param": "petal length (cm)",
        "value": 2.45,
    },
    "node_1": {
        "param": "petal width (cm)",
        "value": 1.75,
    },
    "node_2": {
        "a": {
            "param": "petal length (cm)",
            "value": 4.95,
        },
        "b": {
            "param": "petal length (cm)",
            "value": 4.85,
        }
    },
    "node_3": {
        "a": {
            "param": "petal width (cm)",
            "value": 1.65,
        },
        "b": {
            "param": "petal width (cm)",
            "value": 1.55,
        },
        "c": {
            "param": "sepal width (cm)",
            "value": 3.1,
        }
    },
}


### Declarando a função de de decisão

In [48]:
def get_classification(data, tree):
    data["target_tree1"] = 0    #nó raiz
    for index, row in data.iterrows():
        if not (row[tree["root"]["param"]] <= tree["root"]["value"]):
            data.at[index, "target_tree1"] = 1

    data_node1 = data[data["target_tree1"] == 1]  # separando os dados para o nó1
    
    for index, row in data_node1.iterrows():
        if not (row[tree["node_1"]["param"]] <= tree["node_1"]["value"]):
            data.at[index, "target_tree1"] = 2              # dados para os nós de nível 2
            data_node1.at[index, "target_tree1"] = 2        # usando o index para mudar a classificação no df original

    data_node2a = data_node1[data_node1["target_tree1"] == 1]       # separando a ramificação a e b de nível 2
    data_node2b = data_node1[data_node1["target_tree1"] == 2]
    
    for index, row in data_node2a.iterrows():
        if not (row[tree["node_2"]["a"]["param"]] <= tree["node_2"]["a"]["value"]):
            data.at[index, "target_tree1"] = 2
            data_node2a.at[index, "target_tree1"] = 2
    for index, row in data_node2b.iterrows():
        if not (row[tree["node_2"]["b"]["param"]] <= tree["node_2"]["b"]["value"]):
            data.at[index, "target_tree1"] = 2
            data_node2b.at[index, "target_tree1"] = 2
        else:
            data.at[index, "target_tree1"] = 3              # neste bloco, criamos uma classificacão temporária 3, mas semanticamente, seria igual a 2.
            data_node2b.at[index, "target_tree1"] = 3       # Foi feito pois estes dados serão utilizados no nó 3c, por isso  foi preciso separá-los das outras virgínicas

    data_node3a = data_node2a[data_node2a["target_tree1"] == 1]  #separando os dados para o nó 3a
    data_node3b = data_node2a[data_node2a["target_tree1"] == 2]  #separando os dados para o nó 3b
    
    # nos blocos a seguir, n há a necessidade de mudar as classificaçoes no df dos nós pois os próximos terminais são leaf
    for index, row in data_node3a.iterrows():
        if not (row[tree["node_3"]["a"]["param"]] <= tree["node_3"]["a"]["value"]):
            data.at[index, "target_tree1"] = 2
    for index, row in data_node3b.iterrows():
        if not (row[tree["node_3"]["b"]["param"]] <= tree["node_3"]["b"]["value"]):
            data.at[index, "target_tree1"] = 1

    data_node3c = data_node2b[data_node2b["target_tree1"] == 3]     #separando os dados para o nó 3c
    for index, row in data_node3c.iterrows():
        if not (row[tree["node_3"]["c"]["param"]] <= tree["node_3"]["c"]["value"]):
            data.at[index, "target_tree1"] = 1
        else:
            data.at[index, "target_tree1"] = 2

    return data


In [49]:
dados = get_classification(dados, nodes_tree_1)


## Modelo de Árvore 2
O diagrama da árvore 2 pode ser encontrado no gitbook da proposta de projeto.
Importante ressaltar, que ela é parte do modelo 1.
### Nós
Novamente, declarando os nós em um python dictionary.

A árvore foi dividida em níveis, cima para baixo, esquerda para direita:
1. root
2. node_1
   * leaf
   * node
3. leafs

In [50]:
nodes_tree_2 = {
    "root": {
        "param": "petal length (cm)",
        "value": 2.45,
    },
    "node_1": {
        "param": "petal width (cm)",
        "value": 1.75,
    }
}


Declarando a função

In [51]:
def get_classification2(data, tree=nodes_tree_2):
    data["target_tree2"] = 0

    for index, row in data.iterrows():
        if not (row[tree["root"]["param"]] <= tree["root"]["value"]):
            data.at[index, "target_tree2"] = 1

    data_node1 = data[data["target_tree2"] == 1]
    for index, row in data.iterrows():
        if not (row[tree["node_1"]["param"]] <= tree["node_1"]["value"]):
            data.at[index, "target_tree2"] = 2

    return data

In [52]:
get_classification2(dados)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,target_tree1,target_tree2
0,5.1,3.5,1.4,0.2,0.0,0,0
1,4.9,3.0,1.4,0.2,0.0,0,0
2,4.7,3.2,1.3,0.2,0.0,0,0
3,4.6,3.1,1.5,0.2,0.0,0,0
4,5.0,3.6,1.4,0.2,0.0,0,0
...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2.0,2,2
146,6.3,2.5,5.0,1.9,2.0,2,2
147,6.5,3.0,5.2,2.0,2.0,2,2
148,6.2,3.4,5.4,2.3,2.0,2,2


declarando a função de substituir os números [0, 1, 2] pelos nomes dos targets:

In [53]:
def get_iris_species(x):
    return target_names[x]


## Verificando a performance dos modelos

Será necessário mudar o tipo float do target para int como o objetivo de usá-lo com índice da lista ```target_names```.

In [54]:
dados["target"] = dados["target"].astype(int)
dados["target"] = dados["target"].apply(get_iris_species)
dados["target_tree1"] = dados["target_tree1"].apply(get_iris_species)
dados["target_tree2"] = dados["target_tree2"].apply(get_iris_species)

Criando duas colunas de erros e acertos da predição dos modelos.

In [55]:
dados["tree1_guess"] = dados["target"] == dados["target_tree1"]
dados["tree2_guess"] = dados["target"] == dados["target_tree2"]


### Contagem de Acerto/Erros.
Modelo 1 - 149/1

Modelo 2 - 144/6

In [56]:
dados["tree1_guess"].value_counts()


True     149
False      1
Name: tree1_guess, dtype: int64

In [57]:
dados["tree2_guess"].value_counts()


True     144
False      6
Name: tree2_guess, dtype: int64

### Dispersão dos erros.
Pontos em vermelho são os ítens que forma classificados errado.

In [58]:
# dados1.to_csv('dadosproj.csv')
fig_scatter_result = px.scatter(dados, x="petal length (cm)", y="petal width (cm)",
                                color="tree1_guess", title="Dispersão de Acertos/Erros Árvore 1")
fig_scatter_result.show()


In [59]:
# dados1.to_csv('dadosproj.csv')
fig_scatter_result = px.scatter(dados, x="petal length (cm)", y="petal width (cm)",
                                color="tree2_guess", title="Dispersão de Acertos/Erros Árvore 2")
fig_scatter_result.show()


É importante ressaltar que os erros do modelo 2 estão em uma região problemática em relação à classificação pois os grupos das medidas das íris versicolor e virgínica se intersecionam como é possível verificar na dispersão abaixo:

In [60]:
fig_scatter = px.scatter(dados, x="petal length (cm)",
                         y="petal width (cm)", color="target")
fig_scatter.show()

### Taxa de acertos dos modelos de árvore

In [61]:
acc_tree1 = dados.loc[dados["tree1_guess"] ==
                      True, 'tree1_guess'].count() / len(dados) * 100
acc_tree2 = dados.loc[dados["tree2_guess"] ==
                      True, 'tree2_guess'].count() / len(dados) * 100
print(
    f'Taxa de acertos Modelo1:.... {acc_tree1:.2f}%\nTaxa de acertos Modelo2:.... {acc_tree2:.2f}%')


Taxa de acertos Modelo1:.... 99.33%
Taxa de acertos Modelo2:.... 96.00%


### Matriz de Confusão

#### Modelo 1

In [62]:
setosa_setosa = 0
setosa_versicolor = 0
setosa_virginica = 0

versicolor_setosa = 0
versicolor_virginica = 0
versicolor_versicolor = 0

virginica_setosa = 0
virginica_virginica = 0
virginica_versicolor = 0

for index, row in dados.iterrows():
    if row["target"] == "setosa":
        if row["target_tree1"] == "setosa":
            setosa_setosa +=1
        if row["target_tree1"] == "versicolor":
            setosa_versicolor +=1
        if row["target_tree1"] == "virginica":
            setosa_virginica +=1
        
    if row["target"] == "versicolor":
        if row["target_tree1"] == "setosa":
            versicolor_setosa +=1
        if row["target_tree1"] == "versicolor":
            versicolor_versicolor +=1
        if row["target_tree1"] == "virginica":
            versicolor_virginica +=1
        
    if row["target"] == "virginica":
        if row["target_tree1"] == "versicolor":
            virginica_versicolor +=1
        if row["target_tree1"] == "virginica":
            virginica_virginica +=1
        if row["target_tree1"] == "setosa":
            virginica_setosa +=1

tree1_counts = [
        [setosa_setosa, setosa_virginica, setosa_versicolor],
        [virginica_setosa, virginica_virginica, virginica_versicolor],
        [versicolor_setosa, versicolor_virginica, versicolor_versicolor]
    ]
print(tree1_counts)

[[50, 0, 0], [0, 49, 1], [0, 0, 50]]


A matriz de confusão é composta pelas classes esperadas o ```target```, enquanto que a coluna à esquerda, temos a classificação dada pelo algoritmo.

A linha diagonal em amarelo representa a quantidade de flores que foram corretamente clasificadas, enquanto que em lilás, temos a quantidade de flores que foram incorretamente classificadas. Temos na matriz, uma tipo virginica classificada como versicolor.

In [63]:
tree1_matrix = pd.DataFrame(tree1_counts, columns=target_names, index=target_names)
tree1_matrix.style.background_gradient(cmap='viridis')\
    .set_properties(**{'font-size': '20px'})

,setosa,versicolor,virginica
setosa,50,0,0
versicolor,0,49,1
virginica,0,0,50


#### Modelo 2

In [64]:
setosa_setosa = 0
setosa_versicolor = 0
setosa_virginica = 0

versicolor_setosa = 0
versicolor_virginica = 0
versicolor_versicolor = 0

virginica_setosa = 0
virginica_virginica = 0
virginica_versicolor = 0

for index, row in dados.iterrows():
    if row["target"] == "setosa":
        if row["target_tree2"] == "setosa":
            setosa_setosa += 1
        if row["target_tree2"] == "versicolor":
            setosa_versicolor += 1
        if row["target_tree2"] == "virginica":
            setosa_virginica += 1

    if row["target"] == "versicolor":
        if row["target_tree2"] == "setosa":
            versicolor_setosa += 1
        if row["target_tree2"] == "versicolor":
            versicolor_versicolor += 1
        if row["target_tree2"] == "virginica":
            versicolor_virginica += 1

    if row["target"] == "virginica":
        if row["target_tree2"] == "versicolor":
            virginica_versicolor += 1
        if row["target_tree2"] == "virginica":
            virginica_virginica += 1
        if row["target_tree2"] == "setosa":
            virginica_setosa += 1

tree2_counts = [
        [setosa_setosa, setosa_virginica, setosa_versicolor],
        [virginica_setosa, virginica_virginica, virginica_versicolor],
        [versicolor_setosa, versicolor_virginica, versicolor_versicolor]
    ]
print(tree2_counts)


[[50, 0, 0], [0, 45, 5], [0, 1, 49]]


Abaixo, na matriz de confusão do Modelo 2, é possível notar que tivemos 5 íris virginicas classificadas erroneamente como versicolor e uma versicolor classificada erroneamente como virginica.

In [65]:
tree2_matrix = pd.DataFrame(tree2_counts, columns=target_names, index=target_names)
tree2_matrix.style.background_gradient(cmap='viridis')\
    .set_properties(**{'font-size': '20px'})

,setosa,versicolor,virginica
setosa,50,0,0
versicolor,0,45,5
virginica,0,1,49


## Considerações finais
Conforme demonstrado, a taxa de acertos do Modelo 1 é maior que a do Modelo 2 (99.33% x 96%), o que nos leva a afirmar que o Modelo 1 é melhor do que o Modelo 2. Entretanto, é importante ressaltar que o Modelo2 é um modelo simples, com os nós intuitivos, bem perceptíveis quando examinamos a dispersão *petal length* x *petal width* enquanto que o Modelo 1 é o Modelo 2 com mais *branches*, tornando-o mais complexo e profundo. Seria necessário outros parâmetros para avaliar de forma mais detalhada o desempenho dos modelos, porém, infelizmente o autor deste notebook não dispõe de tais ferramentas.

Considerando o exposto, o melhor modelo é o Modelo 1, tendo com parâmetro, a taxa de acertos.